<a href="https://colab.research.google.com/github/syedmahmoodiagents/Basics/blob/main/LangChain_Session.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain-huggingface --q

In [2]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

In [3]:
llm_hg = ChatHuggingFace(llm=HuggingFaceEndpoint(repo_id='meta-llama/Llama-3.1-8B-Instruct'))

In [4]:
import os, getpass

In [5]:
os.environ['HF_TOKEN'] = getpass.getpass()

··········


In [6]:
llm_hg.invoke("what is the capital of France?")

AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 42, 'total_tokens': 50}, 'model_name': 'meta-llama/Llama-3.1-8B-Instruct', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019ba135-17c3-7153-a0bd-8d23168bcc20-0', usage_metadata={'input_tokens': 42, 'output_tokens': 8, 'total_tokens': 50})

In [17]:
from langchain_core.chat_history import InMemoryChatMessageHistory

In [18]:
mem = InMemoryChatMessageHistory()

In [19]:
mem.add_user_message("what is the capital of France?")
mem.add_ai_message("Paris haha hehe")
mem.add_user_message("what is the capital of Germany?")
mem.add_ai_message("Berlin haha hehe")
mem.add_user_message("By taking the examples from above answers tell us what is the capital of Italy? exactly the same way how its given in example?")

In [20]:
mem

InMemoryChatMessageHistory(messages=[HumanMessage(content='what is the capital of France?', additional_kwargs={}, response_metadata={}), AIMessage(content='Paris haha hehe', additional_kwargs={}, response_metadata={}), HumanMessage(content='what is the capital of Germany?', additional_kwargs={}, response_metadata={}), AIMessage(content='Berlin haha hehe', additional_kwargs={}, response_metadata={}), HumanMessage(content='By taking the examples from above answers tell us what is the capital of Italy? exactly the same way how its given in example?', additional_kwargs={}, response_metadata={})])

In [21]:
mem.messages

[HumanMessage(content='what is the capital of France?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Paris haha hehe', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is the capital of Germany?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Berlin haha hehe', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='By taking the examples from above answers tell us what is the capital of Italy? exactly the same way how its given in example?', additional_kwargs={}, response_metadata={})]

In [22]:
llm_hg.invoke(mem.messages)

AIMessage(content='Rome haha hehe', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 103, 'total_tokens': 109}, 'model_name': 'meta-llama/Llama-3.1-8B-Instruct', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019ba13c-51c8-77a1-a496-94c9f0f58fc6-0', usage_metadata={'input_tokens': 103, 'output_tokens': 6, 'total_tokens': 109})

In [28]:
store = {}

In [32]:
store['David'] = InMemoryChatMessageHistory()

In [33]:
store['Harry'] = InMemoryChatMessageHistory()

In [34]:
store['David'].add_user_message("what is spoken in France?")
store['David'].add_ai_message("French haha hehe?")
store['David'].add_user_message("what is spoken in Italy?")
store['David'].add_ai_message("Italian haha hehe?")
store['David'].add_user_message("what is spoken in Germany?")

In [35]:
llm_hg.invoke(store['David'].messages)

AIMessage(content='German.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 83, 'total_tokens': 86}, 'model_name': 'meta-llama/Llama-3.1-8B-Instruct', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019ba142-5381-7cb0-887e-ceaf0e5ce776-0', usage_metadata={'input_tokens': 83, 'output_tokens': 3, 'total_tokens': 86})

# Real Session

In [37]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [41]:
from langchain_core.runnables import RunnableWithMessageHistory

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory

In [38]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are an AI assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

In [53]:
store = {}  # store sessions here
def get_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [54]:
app = RunnableWithMessageHistory(
    runnable=template | llm_hg,
    get_session_history=get_history,
    input_messages_key="input",
    history_messages_key="history"
)

In [55]:
get_history('David').add_user_message("what is spoken in France?")
get_history('David').add_ai_message("French haha hehe?")
get_history('David').add_user_message("what is spoken in Italy?")
get_history('David').add_ai_message("Italian haha hehe?")

In [56]:
get_history('David')

InMemoryChatMessageHistory(messages=[HumanMessage(content='what is spoken in France?', additional_kwargs={}, response_metadata={}), AIMessage(content='French haha hehe?', additional_kwargs={}, response_metadata={}), HumanMessage(content='what is spoken in Italy?', additional_kwargs={}, response_metadata={}), AIMessage(content='Italian haha hehe?', additional_kwargs={}, response_metadata={})])

In [48]:
# app.invoke({"input": "What did I just say?"}, config={"configurable": {"session_id": "David"}})

AIMessage(content='You asked about the languages spoken in different countries, and I answered with the languages spoken in France and Italy. You seemed to be expecting a repetition of the question, so I repeated the country names and their languages, but in a playful way (with "haha hehe").', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 89, 'total_tokens': 146}, 'model_name': 'meta-llama/Llama-3.1-8B-Instruct', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019ba163-e423-7d72-82f4-ff763b259bf7-0', usage_metadata={'input_tokens': 89, 'output_tokens': 57, 'total_tokens': 146})

In [57]:
resp = app.invoke({"input": "What did I just say?"}, config={"configurable": {"session_id": "David"}})

In [58]:
resp

AIMessage(content='You asked what languages are spoken in France and Italy.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 89, 'total_tokens': 101}, 'model_name': 'meta-llama/Llama-3.1-8B-Instruct', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019ba166-6249-76f0-80f1-84f89890a9e8-0', usage_metadata={'input_tokens': 89, 'output_tokens': 12, 'total_tokens': 101})

In [59]:
resp.content

'You asked what languages are spoken in France and Italy.'